## Загрузим нужные библиотеки

In [1]:
import os
import re
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor, Pool


SEED = 0
VERSION = "catboost_cv5-3"
FOLDS   = 5

%matplotlib inline

## Зафиксируем генератор случайных чисел

In [2]:
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(SEED)

## Загрузим тренировочный датасет

In [3]:
df = pd.read_csv("train_dataset_train.csv").drop_duplicates()

In [4]:
df.head(3)

,id,Дата,Время,Место,Улица,Дом,Дорога,Километр,Метр,Вид ДТП,Погибло,Погибло детей,Ранено,Ранено детей
0,490103984,13.07.2018,17:35:00,"Новгородская область, Великий Новгород",Большая Санкт-Петербургская ул,88,NaN,NaN,NaN,Столкновение,0,0,0,0
1,490097169,11.05.2018,17:10:00,"Новгородская область, Великий Новгород",Большая Санкт-Петербургская ул,73,NaN,NaN,NaN,Столкновение,0,0,0,0
2,490031781,25.01.2020,14:44:00,"Новгородская область, Великий Новгород",Хутынская ул,29,NaN,NaN,NaN,NaN,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19996 entries, 0 to 35769
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             19996 non-null  int64  
 1   Дата           19996 non-null  object 
 2   Время          19996 non-null  object 
 3   Место          19996 non-null  object 
 4   Улица          11297 non-null  object 
 5   Дом            11391 non-null  object 
 6   Дорога         5763 non-null   object 
 7   Километр       5764 non-null   float64
 8   Метр           5764 non-null   float64
 9   Вид ДТП        14538 non-null  object 
 10  Погибло        19996 non-null  int64  
 11  Погибло детей  19996 non-null  int64  
 12  Ранено         19996 non-null  int64  
 13  Ранено детей   19996 non-null  int64  
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


## Много пустых значений, обработаем данные

In [6]:
def addr_aplit(x):
    x = str(x).split(',')[1:]
    if len(x) == 1:
        if x[-1].strip().endswith('район'):
            return x[0].strip(), "", ""
        else:
            return "", x[0].strip(), ""
    elif len(x) == 2:
        return x[0].strip(), x[1].strip(), ""
    elif len(x) == 3:
        return x[0].strip(), x[1].strip(), x[2].strip()
    else:
        return "", "", ""

def house_apply(x):
    if isinstance(x, int):
        return x
    if not isinstance(x, str):
        return 0
    x = rep.sub(" ", x)
    x = x.strip().split(' ')[0]
    if x == "":
        return 0
    else:
        return int(x)

In [7]:
df['Вид ДТП'].fillna("-", inplace=True)
df['Метр'].fillna(500.0, inplace=True)

df["Год"]  = df["Дата"].str[6:]
df["День"] = df["Дата"].str[:2]
df["Месяц"]= df["Дата"].str[3:5]
df["Час"]  = df["Время"].str[:2]

df[['Район', 'Город', "Деревня"]] = np.vstack(df['Место'].map(addr_aplit))

## Разделим датасет на 2 части: улица + дом или дорога + километр + метр и обучим 2 модели catboost

In [8]:
columns_num = ["Год", "Месяц", "День", "Час"]
columns_cat1 = ['Район', 'Город', "Деревня", "Улица", "Вид ДТП"]
columns_cat2 = ['Район', 'Город', "Деревня", "Дорога", "Вид ДТП"]
columns_num1 = ["Дом"]
columns_num2 = ["Километр", "Метр"]
columns_y   = ["Погибло", "Погибло детей", "Ранено", "Ранено детей"]

df[columns_num] = df[columns_num].values.astype(int)
df[columns_y]   = df[columns_y].values.astype(int)

In [9]:
df1 = df[~df['Улица'].isna()]
df1 = df1[df1['Дом'] != 0].reset_index(drop=True)

rep = re.compile("[^\d]")
    
df1['Дом'] = df1['Дом'].map(house_apply)

df1[columns_num + columns_num1 + columns_cat1 + columns_y].head()

,Год,Месяц,День,Час,Дом,Район,Город,Деревня,Улица,Вид ДТП,Погибло,Погибло детей,Ранено,Ранено детей
0,2018,7,13,17,88,,Великий Новгород,,Большая Санкт-Петербургская ул,Столкновение,0,0,0,0
1,2018,5,11,17,73,,Великий Новгород,,Большая Санкт-Петербургская ул,Столкновение,0,0,0,0
2,2020,1,25,14,29,,Великий Новгород,,Хутынская ул,-,0,0,0,0
3,2020,5,11,11,4,Новгородский район,Савинский,д Зарелье,Вишнёвая ул,-,0,0,0,0
4,2019,10,11,18,88,,Великий Новгород,,Большая Санкт-Петербургская ул,Столкновение,0,0,0,0


In [10]:
print(len(df1))
df1[columns_y].sum(axis=0)

11297


Погибло           172
Погибло детей       6
Ранено           3274
Ранено детей      546
dtype: int64

In [11]:
df2 = df[~df['Дорога'].isna()]
df2 = df2[~df2['Километр'].isna()].reset_index(drop=True)
df2[columns_num + columns_cat2+columns_num2 + columns_y].head()

,Год,Месяц,День,Час,Район,Город,Деревня,Дорога,Вид ДТП,Километр,Метр,Погибло,Погибло детей,Ранено,Ранено детей
0,2019,1,9,1,Окуловский район,,,Нева Москва - Санкт-Петербург (основное направ...,Иной вид ДТП,430.0,5.0,0,0,0,0
1,2020,1,14,12,Новгородский район,,,Великий Новгород - Нехино,-,3.0,840.0,0,0,0,0
2,2019,3,1,12,Чудовский район,,,"""Россия"" Москва - Тверь - Великий Новгород - С...",Наезд на препятствие,573.0,700.0,0,0,0,0
3,2019,2,21,2,Маловишерский район,,,Нева Москва - Санкт-Петербург (основное направ...,Наезд на препятствие,474.0,800.0,0,0,0,0
4,2020,5,6,18,Новгородский район,,,Подъезд к городу Новгород от а/д М-10,-,39.0,200.0,0,0,0,0


In [12]:
print(len(df2))
df2[columns_y].sum(axis=0)

5758


Погибло           201
Погибло детей       5
Ранено           2029
Ранено детей      235
dtype: int64

## Разобьем выюорку на 5 фолдов и обучим 5 моделей для первой части данных

In [13]:
seed_everything(SEED)
cv = KFold(n_splits=FOLDS, random_state=SEED, shuffle=True)

val_preds = np.zeros((len(df1), 4))
train_preds = np.zeros((len(df1), 4))

models1 = []

tree_params = {
    'max_depth': 9,
    'eval_metric': 'MultiRMSE',
    'loss_function': 'MultiRMSE',
    'random_state': SEED,
    'iterations': 15_000,
    'l2_leaf_reg': 3,
    'learning_rate': 0.0008,
    'use_best_model': True,
    'task_type': 'CPU',
    'verbose': 1000,
    'cat_features': columns_cat1,
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(df1), 1):
    print(f'Training with fold {fold_} started.')
    model = CatBoostRegressor(**tree_params)    

    train, val = df1[columns_num + columns_num1 + columns_cat1].iloc[train_idx], df1[columns_num + columns_num1 + columns_cat1].iloc[val_idx]

    train_pool = Pool(train, df1[columns_y].iloc[train_idx], cat_features=columns_cat1)
    val_pool = Pool(val, df1[columns_y].iloc[val_idx], cat_features=columns_cat1)

    model.fit(train_pool, eval_set=[val_pool], early_stopping_rounds=500, verbose_eval=1000, use_best_model=True, plot=False)

    val_preds[val_idx] = model.predict(val_pool)
    train_preds[train_idx] += model.predict(train_pool) / (cv.n_splits-1)
    models1.append(model)
    print(f'Training with fold {fold_} completed.')


Training with fold 1 started.
0:	learn: 0.7064006	test: 0.7592942	best: 0.7592942 (0)	total: 136ms	remaining: 34m 3s
1000:	learn: 0.6979544	test: 0.7546860	best: 0.7546860 (1000)	total: 1m 20s	remaining: 18m 45s
2000:	learn: 0.6930273	test: 0.7531369	best: 0.7531369 (2000)	total: 2m 46s	remaining: 18m 4s
3000:	learn: 0.6890674	test: 0.7522479	best: 0.7522479 (3000)	total: 4m 20s	remaining: 17m 22s
4000:	learn: 0.6856220	test: 0.7516420	best: 0.7516411 (3995)	total: 6m 1s	remaining: 16m 34s
5000:	learn: 0.6823272	test: 0.7511567	best: 0.7511555 (4998)	total: 7m 46s	remaining: 15m 32s
6000:	learn: 0.6792553	test: 0.7508142	best: 0.7508142 (6000)	total: 9m 29s	remaining: 14m 13s
7000:	learn: 0.6763296	test: 0.7505373	best: 0.7505371 (6999)	total: 11m 15s	remaining: 12m 51s
8000:	learn: 0.6734367	test: 0.7503476	best: 0.7503454 (7907)	total: 13m	remaining: 11m 22s
9000:	learn: 0.6705175	test: 0.7502208	best: 0.7502153 (8955)	total: 14m 50s	remaining: 9m 53s
10000:	learn: 0.6673814	test: 0.

## Разобьем выборку на 5 фолдов и обучим 5 моделей для второй части данных

In [14]:
seed_everything(SEED)
cv = KFold(n_splits=FOLDS, random_state=SEED, shuffle=True)

val_preds = np.zeros((len(df2), 4))
train_preds = np.zeros((len(df2), 4))

models2 = []

tree_params = {
    'max_depth': 9,
    'eval_metric': 'MultiRMSE',
    'loss_function': 'MultiRMSE',
    'random_state': SEED,
    'iterations': 15_000,
    'l2_leaf_reg': 3,
    'learning_rate': 0.0012,
    'use_best_model': True,
    'task_type': 'CPU',
    'verbose': 1000,
    'cat_features': columns_cat2,
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(df2), 1):
    print(f'Training with fold {fold_} started.')
    model = CatBoostRegressor(**tree_params)    

    train, val = df2[columns_num + columns_cat2 + columns_num2].iloc[train_idx], df2[columns_num + columns_cat2 + columns_num2].iloc[val_idx]

    train_pool = Pool(train, df2[columns_y].iloc[train_idx], cat_features=columns_cat2)
    val_pool = Pool(val, df2[columns_y].iloc[val_idx], cat_features=columns_cat2)

    model.fit(train_pool, eval_set=[val_pool], early_stopping_rounds=500, verbose_eval=1000, use_best_model=True, plot=False)

    val_preds[val_idx] = model.predict(val_pool)
    train_preds[train_idx] += model.predict(train_pool) / (cv.n_splits-1)
    models2.append(model)
    print(f'Training with fold {fold_} completed.')


Training with fold 1 started.
0:	learn: 0.8067098	test: 0.7620466	best: 0.7620466 (0)	total: 74.6ms	remaining: 18m 39s
1000:	learn: 0.7861691	test: 0.7586015	best: 0.7586008 (996)	total: 1m 13s	remaining: 17m 8s
2000:	learn: 0.7714062	test: 0.7575630	best: 0.7575563 (1985)	total: 2m 26s	remaining: 15m 54s
3000:	learn: 0.7582703	test: 0.7572044	best: 0.7572044 (3000)	total: 3m 40s	remaining: 14m 40s
4000:	learn: 0.7467283	test: 0.7570738	best: 0.7570716 (3937)	total: 4m 54s	remaining: 13m 30s
5000:	learn: 0.7356813	test: 0.7570819	best: 0.7570350 (4584)	total: 6m 10s	remaining: 12m 20s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.7570350019
bestIteration = 4584

Shrink model to first 4585 iterations.
Training with fold 1 completed.
Training with fold 2 started.
0:	learn: 0.7976852	test: 0.7989988	best: 0.7989988 (0)	total: 88.3ms	remaining: 22m 4s
1000:	learn: 0.7827243	test: 0.7956718	best: 0.7956716 (999)	total: 1m 15s	remaining: 17m 29s
2000:	learn: 0.7711273	

## Загрузим тестовые данные и проведем над ними такие-же преобразования

In [15]:
df_test = pd.read_csv('test_dataset_test.csv')
df_samp = pd.read_csv('sample_solution.csv')

df_test['Вид ДТП'].fillna("-", inplace=True)
df_test['Километр']  = df_test['Километр'].fillna(0.0).values.astype(float)
df_test['Метр']      = df_test['Метр'].fillna(500.0).values.astype(float)
df_test['Дистанция'] = df_test['Километр'] * 1000 + df_test['Метр']

df_test["Год"]  = df_test["Дата"].str[6:]
df_test["День"] = df_test["Дата"].str[:2]
df_test["Месяц"]= df_test["Дата"].str[3:5]
df_test["Час"]  = df_test["Время"].str[:2]

df_test[['Район', 'Город', "Деревня"]] = np.vstack(df_test['Место'].map(addr_aplit))

df_test[columns_num] = df_test[columns_num].values.astype(int)
df_test[columns_num2] = df_test[columns_num2].values.astype(float)

df_test1 = df_test[~df_test['Улица'].isna()]
df_test1 = df_test1[~df_test1['Дом'].isna()]

df_test1['Дом'] = df_test1['Дом'].map(house_apply)

df_test2 = df_test[~df_test['Дорога'].isna()]
df_test2 = df_test2[~df_test2['Километр'].isna()]

In [16]:
df_test1[columns_num + columns_num1 + columns_cat1]

,Год,Месяц,День,Час,Дом,Район,Город,Деревня,Улица,Вид ДТП
0,2018,1,31,19,59,,Великий Новгород,,Нехинская ул,Столкновение
1,2019,6,19,10,38,,Великий Новгород,,Большая Московская ул,Столкновение
3,2019,2,1,13,3,,Великий Новгород,,Колмовская наб,Наезд на препятствие
6,2018,2,20,13,80,,Великий Новгород,,Черняховского ул,Столкновение
7,2020,8,21,15,25,Боровичский район,Боровичи,,Парковая ул,-
...,...,...,...,...,...,...,...,...,...,...
6781,2019,7,29,13,28,,Великий Новгород,,Большая Санкт-Петербургская ул,Столкновение
6782,2018,11,13,17,69,,Великий Новгород,,Псковская ул,Наезд на пешехода
6783,2020,7,11,23,1,Боровичский район,Боровичи,,Коммунарная ул,-
6786,2018,11,11,16,53,Окуловский район,Окуловка,,Николая Николаева ул,Столкновение


## Делаем прогнозы для первой и второй части датасета и потом склеиваем в единый датафрейм

In [17]:
df_test1[columns_y] = np.zeros((len(df_test1), 4), dtype=float)
df_test2[columns_y] = np.zeros((len(df_test2), 4), dtype=float)

In [18]:
for model in models1:
    df_test1[columns_y] += (model.predict(df_test1[columns_num + columns_num1 + columns_cat1]) + 0.5) / len(models1)

df_test1[columns_y] = df_test1[columns_y].values.astype(int)
df_test1[columns_y].sum()

Погибло            0
Погибло детей      0
Ранено           203
Ранено детей       0
dtype: int64

In [19]:
for model in models2:
    df_test2[columns_y] += (model.predict(df_test2[columns_num + columns_cat2 + columns_num2]) + 0.5) / len(models2)

df_test2[columns_y] = df_test2[columns_y].values.astype(int)
df_test2[columns_y].sum()

Погибло           0
Погибло детей     0
Ранено           92
Ранено детей      0
dtype: int64

In [20]:
df_samp = pd.merge(df_samp[['id']], df_test1.append(df_test2)[['id'] + columns_y], how='left', left_on='id', right_on='id').fillna(0.0)
df_samp[columns_y] = df_samp[columns_y].astype(int)
df_samp.to_csv(f'{VERSION}_{SEED}.csv', index=False)

df_samp.head()

/tmp/ipykernel_1097/3211576138.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_samp = pd.merge(df_samp[['id']], df_test1.append(df_test2)[['id'] + columns_y], how='left', left_on='id', right_on='id').fillna(0.0)


,id,Погибло,Погибло детей,Ранено,Ранено детей
0,490078911,0,0,0,0
1,490055448,0,0,0,0
2,490054440,0,0,0,0
3,490037847,0,0,0,0
4,490037049,0,0,0,0
